# Data processing


author: steeve.laquitaine@epfl.ch  
date: 2023.03.02  
last modified: 2023.10.18  
status:  OK  
display-status: up-to-date  
regression: None  


## Setup

Activate jupyter kernel from python virtual env. `npx_10m_384ch_unit_classes.txt`

In [32]:
 # listen to changes
%load_ext autoreload
%autoreload 2

import os

# set project path
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/sfn_2023/"

os.chdir(PROJ_PATH)

from src.nodes.utils import get_config
from src.nodes.load import load_campaign_params
from src.nodes.dataeng.silico import recording, probe_wiring
from src.nodes.prepro import preprocess
from src.nodes.truth.silico import ground_truth
from src.nodes.dataeng.lfp_only import stacking

# SETUP PARAMETERS
EXPERIMENT = "silico_neuropixels"   # specifies the experiment 
SIMULATION_DATE = "2023_02_19"      # specifies the run (date)

data_conf, param_conf = get_config(EXPERIMENT, SIMULATION_DATE).values()

GAIN = 1e5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2023-10-19 11:42:52,207 - root - utils.py - get_config - INFO - Reading experiment config.
2023-10-19 11:42:52,253 - root - utils.py - get_config - INFO - Reading experiment config. - done


### Create campaign trace and spike files

was done with app.py. We will rerun that experiment with the BlueConfig mosaic tag to only get lfp. For now we process the existing file for speed.

### Rescale, offset, cast as Spikeinterface Recording Extractor object


Traces need rescaling as the simulation produces floats with values concentraed bwteeen -1 and 1. As traces are converted to int16 to be used by Kilosort 3.0, nearly all spikes disappear (set to 0).

return_scale=True does not seem to work as default so we have to rewrite the traces with the new 

In [ ]:
# takes 28 mins

# cast (30 secs)
RecordingExtr = recording.run(data_conf, gain=GAIN, offset=True)

# write (2 mins)
recording.write(RecordingExtr, data_conf)
RecordingExtr = recording.load(data_conf)

In [ ]:
# check is probe
try: 
    RecordingExtr.get_probe() 
except: 
    print("there is no probe wired")

### Wire probe to recording

In [ ]:
# takes 26 min
# run and write
Recording = probe_wiring.run(data_conf, param_conf)
probe_wiring.write(Recording, data_conf)

### Preprocess recording

In [ ]:
# takes 32 min

# preprocess, write
Preprocessed = preprocess.run(data_conf, param_conf)
preprocess.write(Preprocessed, data_conf)

# sanity check is preprocessed
print(Preprocessed.is_filtered())

### Get ground truth spikes
Cast spikes as a SpikeInterface Sorting Extractor object.

In [ ]:
# check write path
print(data_conf["sorting"]["simulation"]["ground_truth"]["output"])

In [ ]:
# takes (1.5h for 534 units)

# get simulation parameters
simulation = load_campaign_params(data_conf)

# cast ground truth spikes as a SpikeInterface Sorting Extractor object 
SortingTrue = ground_truth.run(simulation, data_conf, param_conf)

# write
ground_truth.write(SortingTrue["ground_truth_sorting_object"], data_conf)